In [ ]:
%pip install yandex-cloud-ml-sdk --upgrade

In [ ]:
import pathlib 
import uuid
import os
from __future__ import annotations

from yandex_cloud_ml_sdk import YCloudML
import urllib.request
import zipfile

In [ ]:
def local_path(path: str) -> pathlib.Path:
    return pathlib.Path(path)

dataset_path = local_path("generations.jsonlines")
print(dataset_path)

print("Данные датасета для дообучения")
print(dataset_path.read_text())

In [ ]:
def download_tensorboard(url):
    urllib.request.urlretrieve(url, "tensorboard.zip")
    with zipfile.ZipFile('tensorboard.zip', 'r') as zip_ref:
        zip_ref.extractall("tensorboard")

In [ ]:
# Создаем объект SDK, содержащий параметры для авторизации
# Авторизация происходит от имени сервисного аккаунта с ролью ai.editor
# Для аутентификации используется API-ключ 

sdk = YCloudML(
    folder_id=os.environ['FOLDER_ID'], # Идентификатор каталога, сохраненный в секрете DataSphere
    auth=os.environ['API_KEY'] # API-ключ сервисного аккаунта, сохраненный в секрете DataSphere
)

In [ ]:
print('Создаем датасет для дообучения')

# Создаем датасет для дообучения модели генерации текса
# Указываем путь к файлу с данными, формат и тип датасета
dataset_draft = sdk.datasets.draft_from_path(
    task_type='TextToTextGeneration',
    path=dataset_path,
    upload_format='jsonlines',
    name='test-generations'
)

# Запускаем загрузку и валидацию датасета, дожидаемся окончания
dataset = dataset_draft.upload()
print('Датасет загружается и валидируется')
print(f'Новый датасет {dataset=} \n')

In [ ]:
# Выбираем базовую модель, которую хотим дообучить. В примере — yandexgpt lite
base_model = sdk.models.completions('yandexgpt-lite')

# Запускаем дообучение модели
tuning_task = base_model.tune_deferred(
    dataset,
    name=str(uuid.uuid4()),
    n_samples=10000 
)

print(f'Обучение началось {tuning_task} \n')

# Дообучение может длиться до нескольких часов
# Дожидаемся завершения операции обучения и получаем новую модель
new_model = tuning_task.wait()
print(f'Обучение закончилось, новая модель = {new_model} \n')

In [ ]:
# Если вы закрыли ноутбук, по task_id можно проверить статус операции
tuning_task = sdk.tuning.get(task_id)
print(tuning_task.get_task_info())

# И получить дообученную модель
new_model = tuning_task.get_result()
print({new_model})

In [ ]:
print('Посылаем запросы в новую модель\n')
# Примеры запросов в модель
completion_result = new_model.run("Как тебя зовут?")
print(f'{completion_result=} \n')

# Дообученную модель можно использовать, указав ее URI
tuned_uri = new_model.uri 
model = sdk.models.completions(tuned_uri)

completion_result = model.run("Откуда ты?")
print(f'{completion_result=} \n')

In [ ]:
# Можно посмотреть метрики дообучения
# Получаем ссылку с данными для TensorBoard и скачиваем файлы
metrics_url = new_model.get_metrics_url()
download_tensorboard(metrics_url)